# Import and Read data


In [1]:
import sys
sys.path.append('.')  # Add current directory to path
from build_graph_and_train import *

/opt/conda/lib/python3.11/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
partition = 478

In [3]:
df = pd.read_csv(f"../../../../data/top30groups/DNDF_OneHotLongLatCombined/combined/combined{partition}.csv")

In [4]:
import os 
if not os.path.isdir(f"Results{partition}"):
    os.mkdir(f"Results{partition}")

# Create longlat feature

In [5]:
geodata = ['longitude', 'latitude']
combined_geo = df.copy()
combined_geo['longlat'] = list(zip(df['longitude'], df['latitude']))
combined_geo = combined_geo.drop(columns=geodata)

In [6]:
import ast

def to_tuple_if_needed(val):
    if isinstance(val, str):
        return ast.literal_eval(val)
    return val  # already a tuple

combined_geo['longlat'] = combined_geo['longlat'].apply(to_tuple_if_needed)

In [7]:
combined_geo.dtypes

extended         int64
vicinity         int64
multiple       float64
success          int64
suicide          int64
attacktype1      int64
targtype1        int64
target1          int64
individual       int64
weaptype1        int64
nkill          float64
property         int64
ishostkid      float64
gname           object
longlat         object
dtype: object

# Weapon type prediction

In [8]:
label_index = {g: i for i, g in enumerate(sorted(df['gname'].unique()))}
continuous_col = 'weaptype1'
data, y_gcn, y_nrf, non_geo_features, train_mask, test_mask, row_to_node_index = build_graph_data(combined_geo, label_index, continuous_col=continuous_col)
args = {
    'partition': f"gtd{partition}",
    'embed_dim': 16,
    'lr': 0.01,
    'epochs': 1500,
    'feat_dropout': 0,
    'n_tree': 80,
    'tree_depth': 10,
    'tree_feature_rate': 0.5,
    'n_class': len(label_index)  
}
best_acc, best_epoch = train_joint(data, data.edge_index, y_gcn, y_nrf, non_geo_features, train_mask, test_mask, args, row_to_node_index)

with open(f"Results{partition}/Results_{continuous_col}_prediction", "w") as f:
    f.write(f"Best acc: {best_acc} in epoch {best_epoch} for {continuous_col} prediction")

/home/jovyan/buggy_GTD/GTD_2025/Codes/Baselines/graphrfi/GraphRfi_OneHotLongLat/build_graph_and_train.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_nrf = torch.tensor(y_nrf, dtype=torch.long).to(device)


Epoch 01 | GCN MSE Loss: 838.4142 | NRF Loss: 3.4033 | JOINT Loss: 841.8176 | NRF Acc: 0.0581
Epoch 02 | GCN MSE Loss: 692.1746 | NRF Loss: 3.3978 | JOINT Loss: 695.5724 | NRF Acc: 0.0380
Epoch 03 | GCN MSE Loss: 567.7802 | NRF Loss: 3.4041 | JOINT Loss: 571.1843 | NRF Acc: 0.0382
Epoch 04 | GCN MSE Loss: 461.2041 | NRF Loss: 3.4054 | JOINT Loss: 464.6096 | NRF Acc: 0.0383
Epoch 05 | GCN MSE Loss: 371.0038 | NRF Loss: 3.4017 | JOINT Loss: 374.4056 | NRF Acc: 0.0207
Epoch 06 | GCN MSE Loss: 295.4906 | NRF Loss: 3.4044 | JOINT Loss: 298.8949 | NRF Acc: 0.0207
Epoch 07 | GCN MSE Loss: 232.9050 | NRF Loss: 3.4050 | JOINT Loss: 236.3100 | NRF Acc: 0.0207
Epoch 08 | GCN MSE Loss: 181.7834 | NRF Loss: 3.4016 | JOINT Loss: 185.1850 | NRF Acc: 0.0207
Epoch 09 | GCN MSE Loss: 140.8738 | NRF Loss: 3.4008 | JOINT Loss: 144.2746 | NRF Acc: 0.0221
Epoch 10 | GCN MSE Loss: 108.6328 | NRF Loss: 3.3986 | JOINT Loss: 112.0313 | NRF Acc: 0.0224
Epoch 11 | GCN MSE Loss: 83.7719 | NRF Loss: 3.3985 | JOINT 

# Fatality rate


In [9]:
label_index = {g: i for i, g in enumerate(sorted(df['gname'].unique()))}
continuous_col = 'weaptype1'
data, y_gcn, y_nrf, non_geo_features, train_mask, test_mask, row_to_node_index = build_graph_data(combined_geo, label_index, continuous_col=continuous_col)
args = {
    'partition': f"gtd{partition}",
    'embed_dim': 16,
    'lr': 0.01,
    'epochs': 1500,
    'feat_dropout': 0,
    'n_tree': 80,
    'tree_depth': 10,
    'tree_feature_rate': 0.5,
    'n_class': len(label_index)  
}
best_acc, best_epoch = train_joint(data, data.edge_index, y_gcn, y_nrf, non_geo_features, train_mask, test_mask, args, row_to_node_index)

with open(f"Results{partition}/Results_{continuous_col}_prediction", "w") as f:
    f.write(f"Best acc: {best_acc} in epoch {best_epoch} for {continuous_col} prediction")

/home/jovyan/buggy_GTD/GTD_2025/Codes/Baselines/graphrfi/GraphRfi_OneHotLongLat/build_graph_and_train.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_nrf = torch.tensor(y_nrf, dtype=torch.long).to(device)


Epoch 01 | GCN MSE Loss: 191.7696 | NRF Loss: 3.4013 | JOINT Loss: 195.1709 | NRF Acc: 0.0650
Epoch 02 | GCN MSE Loss: 120.4657 | NRF Loss: 3.3969 | JOINT Loss: 123.8626 | NRF Acc: 0.0531
Epoch 03 | GCN MSE Loss: 80.2699 | NRF Loss: 3.3977 | JOINT Loss: 83.6676 | NRF Acc: 0.0402
Epoch 04 | GCN MSE Loss: 61.7281 | NRF Loss: 3.3942 | JOINT Loss: 65.1223 | NRF Acc: 0.0400
Epoch 05 | GCN MSE Loss: 57.2432 | NRF Loss: 3.3934 | JOINT Loss: 60.6366 | NRF Acc: 0.0455
Epoch 06 | GCN MSE Loss: 60.1161 | NRF Loss: 3.3908 | JOINT Loss: 63.5069 | NRF Acc: 0.0426
Epoch 07 | GCN MSE Loss: 63.3404 | NRF Loss: 3.3921 | JOINT Loss: 66.7325 | NRF Acc: 0.0367
Epoch 08 | GCN MSE Loss: 62.3216 | NRF Loss: 3.3885 | JOINT Loss: 65.7101 | NRF Acc: 0.0362
Epoch 09 | GCN MSE Loss: 56.0703 | NRF Loss: 3.3853 | JOINT Loss: 59.4556 | NRF Acc: 0.0354
Epoch 10 | GCN MSE Loss: 46.3344 | NRF Loss: 3.3867 | JOINT Loss: 49.7211 | NRF Acc: 0.0332
Epoch 11 | GCN MSE Loss: 35.8446 | NRF Loss: 3.3871 | JOINT Loss: 39.2318 | 

# Target Type

In [10]:
label_index = {g: i for i, g in enumerate(sorted(df['gname'].unique()))}
continuous_col = 'targtype1'
data, y_gcn, y_nrf, non_geo_features, train_mask, test_mask, row_to_node_index = build_graph_data(combined_geo, label_index, continuous_col=continuous_col)
args = {
    'partition': f"gtd{partition}",
    'embed_dim': 16,
    'lr': 0.01,
    'epochs': 1500,
    'feat_dropout': 0,
    'n_tree': 80,
    'tree_depth': 10,
    'tree_feature_rate': 0.5,
    'n_class': len(label_index)  
}
best_acc, best_epoch = train_joint(data, data.edge_index, y_gcn, y_nrf, non_geo_features, train_mask, test_mask, args, row_to_node_index)

with open(f"Results{partition}/Results_{continuous_col}_prediction", "w") as f:
    f.write(f"Best acc: {best_acc} in epoch {best_epoch} for {continuous_col} prediction")

/home/jovyan/buggy_GTD/GTD_2025/Codes/Baselines/graphrfi/GraphRfi_OneHotLongLat/build_graph_and_train.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_nrf = torch.tensor(y_nrf, dtype=torch.long).to(device)


Epoch 01 | GCN MSE Loss: 1278.4412 | NRF Loss: 3.4018 | JOINT Loss: 1281.8429 | NRF Acc: 0.0549
Epoch 02 | GCN MSE Loss: 1144.9897 | NRF Loss: 3.3947 | JOINT Loss: 1148.3844 | NRF Acc: 0.0453
Epoch 03 | GCN MSE Loss: 1021.1631 | NRF Loss: 3.3934 | JOINT Loss: 1024.5565 | NRF Acc: 0.0586
Epoch 04 | GCN MSE Loss: 904.5858 | NRF Loss: 3.3889 | JOINT Loss: 907.9746 | NRF Acc: 0.0643
Epoch 05 | GCN MSE Loss: 799.0338 | NRF Loss: 3.3866 | JOINT Loss: 802.4205 | NRF Acc: 0.0723
Epoch 06 | GCN MSE Loss: 702.5641 | NRF Loss: 3.3861 | JOINT Loss: 705.9502 | NRF Acc: 0.0671
Epoch 07 | GCN MSE Loss: 613.2562 | NRF Loss: 3.3861 | JOINT Loss: 616.6423 | NRF Acc: 0.0645
Epoch 08 | GCN MSE Loss: 532.1519 | NRF Loss: 3.3834 | JOINT Loss: 535.5353 | NRF Acc: 0.0659
Epoch 09 | GCN MSE Loss: 459.9361 | NRF Loss: 3.3833 | JOINT Loss: 463.3194 | NRF Acc: 0.0624
Epoch 10 | GCN MSE Loss: 395.3289 | NRF Loss: 3.3816 | JOINT Loss: 398.7105 | NRF Acc: 0.0564
Epoch 11 | GCN MSE Loss: 337.1822 | NRF Loss: 3.3796 |

# Attack type

In [11]:
label_index = {g: i for i, g in enumerate(sorted(df['gname'].unique()))}
continuous_col = 'attacktype1'
data, y_gcn, y_nrf, non_geo_features, train_mask, test_mask, row_to_node_index = build_graph_data(combined_geo, label_index, continuous_col=continuous_col)
args = {
    'partition': f"gtd{partition}",
    'embed_dim': 16,
    'lr': 0.01,
    'epochs': 1500,
    'feat_dropout': 0,
    'n_tree': 80,
    'tree_depth': 10,
    'tree_feature_rate': 0.5,
    'n_class': len(label_index)  
}
best_acc, best_epoch = train_joint(data, data.edge_index, y_gcn, y_nrf, non_geo_features, train_mask, test_mask, args, row_to_node_index)

with open(f"Results{partition}/Results_{continuous_col}_prediction", "w") as f:
    f.write(f"Best acc: {best_acc} in epoch {best_epoch} for {continuous_col} prediction")

/home/jovyan/buggy_GTD/GTD_2025/Codes/Baselines/graphrfi/GraphRfi_OneHotLongLat/build_graph_and_train.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_nrf = torch.tensor(y_nrf, dtype=torch.long).to(device)


Epoch 01 | GCN MSE Loss: 129.8599 | NRF Loss: 3.4015 | JOINT Loss: 133.2614 | NRF Acc: 0.0568
Epoch 02 | GCN MSE Loss: 68.8077 | NRF Loss: 3.3964 | JOINT Loss: 72.2041 | NRF Acc: 0.0626
Epoch 03 | GCN MSE Loss: 30.0687 | NRF Loss: 3.4006 | JOINT Loss: 33.4693 | NRF Acc: 0.0562
Epoch 04 | GCN MSE Loss: 10.4074 | NRF Loss: 3.4023 | JOINT Loss: 13.8097 | NRF Acc: 0.0403
Epoch 05 | GCN MSE Loss: 6.6244 | NRF Loss: 3.4081 | JOINT Loss: 10.0325 | NRF Acc: 0.0416
Epoch 06 | GCN MSE Loss: 12.9318 | NRF Loss: 3.4067 | JOINT Loss: 16.3385 | NRF Acc: 0.0291
Epoch 07 | GCN MSE Loss: 22.1510 | NRF Loss: 3.4017 | JOINT Loss: 25.5527 | NRF Acc: 0.0541
Epoch 08 | GCN MSE Loss: 28.6909 | NRF Loss: 3.3982 | JOINT Loss: 32.0892 | NRF Acc: 0.0376
Epoch 09 | GCN MSE Loss: 30.4242 | NRF Loss: 3.3948 | JOINT Loss: 33.8189 | NRF Acc: 0.0358
Epoch 10 | GCN MSE Loss: 27.8753 | NRF Loss: 3.3920 | JOINT Loss: 31.2673 | NRF Acc: 0.0569
Epoch 11 | GCN MSE Loss: 22.6368 | NRF Loss: 3.3909 | JOINT Loss: 26.0278 | NRF